In [1]:
import json

from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import (AUC, BinaryAccuracy, FalseNegatives, FalsePositives,
                           Precision, Recall, TrueNegatives, TruePositives)
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

from models import *
from utils import *

In [2]:
NFOLDS = 3
INPUT_PATH = '/unibuc-2019-m2-cv'
OUTPUT_PATH = ''

In [ ]:
batch_size = 32
epochs = 5
learning_rate = 0.0005
model_name = "densenet121_v4"

In [ ]:
X, y = load_dataset(train_labels_path=f'{INPUT_PATH}/train_labels.txt',
                    img_path=f'{INPUT_PATH}/data/data')

In [ ]:
initial_bias = output_bias(y)

In [ ]:
skf = StratifiedKFold(n_splits=NFOLDS, shuffle=True)
split_gen = skf.split(X, y)
fold = 0

In [ ]:
def evaluate_fold(model_klass, train_index, test_index, fold, output_bias=None, path=OUTPUT_PATH):
    print(f"START FOR FOLD {fold}")
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    K.clear_session()
    metrics = [
        TruePositives(name='tp'),
        FalsePositives(name='fp'),
        TrueNegatives(name='tn'),
        FalseNegatives(name='fn'),
        BinaryAccuracy(name='accuracy'),
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(name='auc'),
        f1
    ]
    callbacks = [
        ModelCheckpoint(
            f'{path}/{fold}_checkpoint.h5', monitor='val_auc', save_best_only=True, mode='max'),
        EarlyStopping(monitor='val_auc', patience=3, verbose=1,
                      restore_best_weights=True, mode='max'),
    ]
    model = model_klass(learning_rate=learning_rate,
                        metrics=metrics, output_bias=output_bias)

    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        validation_split=0.15,
                        verbose=1,
                        callbacks=callbacks)
    plot_metrics(history, savefig=f'{path}/metrics{fold}.png')

    scores = model.evaluate(X_test, y_test, batch_size=batch_size)
    metrics = dict(zip(model.metrics_names, scores))
    print(metrics)
    with open(f'{path}/metrics{fold}.json', 'w') as json_file:
        json.dump(metrics, json_file)

    model.save(f'{path}/fold{fold}.h5')

    y_pred = model.predict(X_test, batch_size=batch_size)
    y_pred = parse_predict(y_pred)

    conf_matrix = confusion_matrix(y_test, y_pred)
    print(conf_matrix)

    plot_confusion_matrix(conf_matrix, ['0', '1'],
                          title=f'Confusion Matrix fold {fold}',
                          savefig=f'{path}/conf_matrix_fold{fold}.png')
    run_test_data(model, f'{path}/{fold}.csv',
                  img_path=f'{INPUT_PATH}/data/data')

In [0]:
train_index, test_index = next(split_gen)

In [0]:
evaluate_fold(model_klass=densenet_model,
              train_index=train_index, 
              test_index=test_index, 
              fold=fold,
              output_bias=initial_bias)
fold +=1

In [0]:
train_index, test_index = next(split_gen)

In [0]:
evaluate_fold(model_klass=densenet_model,
              train_index=train_index, 
              test_index=test_index, 
              fold=fold,
              output_bias=initial_bias)
fold +=1

In [0]:
train_index, test_index = next(split_gen)

In [0]:
evaluate_fold(model_klass=densenet_model,
              train_index=train_index, 
              test_index=test_index, 
              fold=fold,
              output_bias=initial_bias)
fold +=1